### a) Métricas de Clasificación

**1.** **Precisión (Precision)**:

- Mide la proporción de predicciones positivas que son correctas.

- Fórmula: Precision = TP / (TP + FP)

   - **TP**: Verdaderos positivos.

   - **FP**: Falsos positivos.

**2.** **Recall (Sensibilidad)**:

- Mide la proporción de positivos reales que son correctamente identificados.

- Fórmula: Recall = TP / (TP + FN)

   - FN: Falsos negativos.

**3.** **F1-Score**:

- Es la media armónica de la precisión y el recall.

- Fórmula: F1 = 2 * (Precision * Recall) / (Precision + Recall)

**4.** **AUC (Área bajo la curva ROC)**:

- Mide la capacidad del modelo para distinguir entre clases.

- Un valor de 1 indica un modelo perfecto, mientras que 0.5 indica un modelo aleatorio.

### b ) Métricas de Regresión

**1.** **RMSE (Root Mean Squared Error)**:

- Mide la diferencia entre los valores predichos y los reales.

- Fórmula: RMSE = sqrt(mean((y_pred - y_true)^2))

**2.** **R² (Coeficiente de Determinación)**:

- Mide cuánta variabilidad de la variable dependiente es explicada por el modelo.

- Un valor de 1 indica un ajuste perfecto, mientras que 0 indica que el modelo no explica nada.

**3.** **MAE (Mean Absolute Error)**:

- Mide el error absoluto promedio entre los valores predichos y los reales.

- Fórmula: MAE = mean(|y_pred - y_true|)

#### Implementación Práctica

- **Dataset**: Wine Quality (winequality-red.csv)
- **Clasificación**: Predecir si un vino es de alta calidad (quality >= 7) o no (quality < 7).

- **Regresión**: Predecir la calidad del vino (quality) como un valor numérico.

**Clasificación**: Métricas de Clasificación

In [2]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

// 1. Crear una sesión de Spark
val spark = SparkSession.builder()
  .appName("Classification Metrics Example")
  .master("local[*]")
  .getOrCreate()

// 2. Cargar el dataset winequality-red
val data = spark.read
  .option("header", "true")
  .option("inferSchema", "true")
  .csv("winequality-red.csv") // Reemplaza con la ruta correcta

spark = org.apache.spark.sql.SparkSession@1730877d
data = [fixed_acidity: double, volatile_acidity: double ... 10 more fields]


[fixed_acidity: double, volatile_acidity: double ... 10 more fields]

In [4]:
import org.apache.spark.sql.functions._
// 3. Crear una columna binaria para clasificación (alta calidad: 1, baja calidad: 0)
val labeledData = data.withColumn("high_quality", when($"quality" >= 7, 1).otherwise(0))

labeledData = [fixed_acidity: double, volatile_acidity: double ... 11 more fields]


[fixed_acidity: double, volatile_acidity: double ... 11 more fields]

In [6]:
// 4. Preparar los datos: Crear un vector de características
val featureColumns = Array(
  "fixed_acidity", "volatile_acidity", "citric_acid", "residual_sugar",
  "chlorides", "free_sulf_dioxide", "total_sulfur_dioxide", "density",
  "pH", "sulphates", "alcohol"
)

val assembler = new VectorAssembler()
  .setInputCols(featureColumns)
  .setOutputCol("features")

val assembledData = assembler.transform(labeledData)

lastException = null
featureColumns = Array(fixed_acidity, volatile_acidity, citric_acid, residual_sugar, chlorides, free_sulf_dioxide, total_sulfur_dioxide, density, pH, sulphates, alcohol)
assembler = VectorAssembler: uid=vecAssembler_372491626417, handleInvalid=error, numInputCols=11
assembledData = [fixed_acidity: double, volatile_acidity: double ... 12 more fields]


[fixed_acidity: double, volatile_acidity: double ... 12 more fields]

In [ ]:
// 5. Dividir los datos en conjuntos de entrenamiento y prueba (80% - 20%)
val Array(trainingData, testData) = assembledData.randomSplit(Array(0.8, 0.2), seed = 42)

In [ ]:
import org.apache.spark.ml.classification.RandomForestClassifier

// 6. Entrenar un modelo de clasificación (Random Forest)
val rf = new RandomForestClassifier()
  .setLabelCol("high_quality")
  .setFeaturesCol("features")
  .setNumTrees(50)
  .setSeed(42)

val model = rf.fit(trainingData)

In [ ]:
// 7. Hacer predicciones sobre el conjunto de prueba
val predictions = model.transform(testData)

In [ ]:
// 8. Calcular métricas de clasificación
val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("high_quality")
  .setPredictionCol("prediction")

val accuracy = evaluator.setMetricName("accuracy").evaluate(predictions)
val precision = evaluator.setMetricName("weightedPrecision").evaluate(predictions)
val recall = evaluator.setMetricName("weightedRecall").evaluate(predictions)
val f1 = evaluator.setMetricName("f1").evaluate(predictions)

println(s"Exactitud (Accuracy): $accuracy")
println(s"Precisión (Precision): $precision")
println(s"Recall: $recall")
println(s"F1-Score: $f1")


In [ ]:
// 9. Detener la sesión de Spark
spark.stop()

### Interpretación de Resultados

- **Clasificación**:

   - **Exactitud (Accuracy)**: El porcentaje de predicciones correctas.

   - **Precisión (Precision)**: La proporción de vinos predichos como de alta calidad que realmente lo son.

    - **Recall**: La proporción de vinos de alta calidad que fueron correctamente identificados.

    - **F1-Score**: Un balance entre precisión y recall.